<a href="https://colab.research.google.com/github/Sensify-Lab/Community-Comm/blob/main/Data/firebase_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Community Comm

### Firebase Preprocessing


In the menu: 'Runtime >> Run All' to run all cells in order


## Setup

In [ ]:
!pip install -U -q PyDrive

import numpy as np
import pandas as pd
import json
from google.colab import files
from datetime import date
from scipy import stats

'''
# for pulling data from google drive:

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials 

'''

'\n# for pulling data from google drive:\n\nfrom pydrive.auth import GoogleAuth\nfrom pydrive.drive import GoogleDrive\nfrom google.colab import auth\nfrom oauth2client.client import GoogleCredentials \n\n'

In [ ]:
from scipy import stats

In [ ]:
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

# id='1qum959PUDAuevThqVhVjLykU1lcqUNKD'
# downloaded = drive.CreateFile({'id': id}) 
# downloaded.GetContentFile('user_log_october_22.json')

'''

Normally, we would read data from the Google Drive, but our json file
seems to be too large for this. We'll use files.upload() to grab it
from local storage instead.

'''


uploaded = files.upload() 

Saving user_log_october_22.json to user_log_october_22.json


In [ ]:
data = next(iter(uploaded.values()))

d = json.loads(data.decode())

df = pd.DataFrame(d)

## Functions

In [ ]:
def describe_articles(json):

  rowData = {
          
          "days_active" : [],
          
          "days_with_article" : [],
          
          "total_articles" : [],
      
          "article_rate" : [],

          "bias" : [],
          
          "left_pct" : [],

          "left_center_pct" : [],

          "center_pct" : [],

          "right_center_pct" : [],

          "right_pct" : []
      
      }

  for user in json:
      
      days = 0
      
      n_articles = 0
      
      has_article = 0

      bias = []
      
      #print(user)
      
      for day in json[user]: 
      
          #print(day)
          
          days += 1
          
          if "articles" in json[user][day]:

              articles_df = pd.DataFrame(json[user][day]["articles"])
            
              has_article += 1
              
              n_articles += len(json[user][day]["articles"])

              bias = bias + list(articles_df.loc["bias",:].values)
              
              

      rowData["days_active"].append(days)
      rowData["days_with_article"].append(has_article)
      rowData["total_articles"].append(n_articles)
      rowData["article_rate"].append(has_article/days)
      rowData["bias"].append(stats.mode(bias)[0])

      if (len(bias)>0):
        rowData["left_pct"].append(float(bias.count("left"))/float(len(bias)))
        rowData["left_center_pct"].append(float(bias.count("left-center"))/float(len(bias)))
        rowData["center_pct"].append(float(bias.count("center"))/float(len(bias)))
        rowData["right_center_pct"].append(float(bias.count("right-center"))/float(len(bias)))
        rowData["right_pct"].append(float(bias.count("right"))/float(len(bias)))

      else:
        rowData["left_pct"].append(0)
        rowData["left_center_pct"].append(0)
        rowData["center_pct"].append(0)
        rowData["right_center_pct"].append(0)
        rowData["right_pct"].append(0)

  # print(rowData)
  df = pd.DataFrame(data=rowData, index=json.keys(),columns=["days_active", "days_with_article", "total_articles","article_rate", "bias","left_pct","left_center_pct","center_pct","right_center_pct", "right_pct"])

  return df

In [ ]:
def describe_browsing(json):

  rowData = {
          
          "total_seconds" : [],

          "avg_sec_per_day" : [],

          "median_sec_per_day" : [],

          "unique_domains" : []

          #"top_domain" : []
      
      }

  for user in json:

    domain_count = pd.DataFrame()

    domains = np.array([], dtype="str")

    seconds = np.array([])

    for date in json[user]: 

      if ("articles" in json[user][date].keys()): user_day_df = pd.DataFrame(json[user][date]).drop("articles", axis=1)

      else: user_day_df = pd.DataFrame(json[user][date])

      seconds = np.append(seconds, user_day_df.loc["seconds", :].sum())

      domains = np.append(domains, user_day_df.loc["website",:].unique())

      domains_df = pd.DataFrame(user_day_df.T)

      domains_df = pd.DataFrame(domains_df[domains_df["pushType"]==0].loc[:,"website"], columns=["website"])

      domains_df[f"{date}"] = 1

      domains_df = domains_df.groupby("website").count()

      if (domain_count.empty): domain_count = domains_df
      else: domain_count = domain_count.join(domains_df, "website").fillna(0)
      # print(domain_count)

    rowData["total_seconds"].append(seconds.sum())

    rowData["avg_sec_per_day"].append(np.mean(seconds))

    rowData["median_sec_per_day"].append(np.median(seconds))

    rowData["unique_domains"].append(len(np.unique(domains.astype("str"))))

    #rowData["top_domain"].append()

    print(seconds)

  df = pd.DataFrame(data=rowData, index=json.keys(), columns = ["total_seconds", "avg_sec_per_day", "median_sec_per_day" ,"unique_domains"])

  return df

In [ ]:
def describe_user_browsing(json, user):

  rowData = {
          
          "total_seconds" : [],

          "unique_domains" : []
      }


  for date in json[user]: 

    seconds = 0

    domains = 0

    user_day_df = pd.DataFrame(json[user][date])

    seconds = user_day_df.loc["seconds", :].sum()

    domains = user_day_df.loc["website",:].unique().size

    rowData["total_seconds"].append(seconds)

    rowData["unique_domains"].append(domains)

  df = pd.DataFrame(data=rowData, index=json[user].keys(), columns = ["total_seconds", "unique_domains"])

  return df

In [ ]:
def collect_articles(json):

  articles = {}

  for user in json:

    user_df = pd.DataFrame(json[user])

    if "articles" in user_df.index:
      #print(user_df.index)
      for i in user_df.loc["articles", :].dropna():

        for k in i.keys():

          if k in articles.keys() and user not in articles[k]:

            articles[k].append(user)

          elif k not in articles.keys():
            articles[k] = [user]

  return articles

In [ ]:
def generate_feature_table(json, exclude=[]):

  filter_accounts(json, exclude) # This function modifies the json argument directly, returns nothing. 

  browsing_df = describe_browsing(json)

  articles_df = describe_articles(json)

  table = browsing_df.join(articles_df)

  table.index.name = "user"

  return table


In [ ]:
def filter_accounts(json, exclude):

  # NOTE : MODIFIES THE JSON DIRECTLY, RETURNS NOTHING

  for i in exclude:
    json.pop(i, None)

## Run & Export

In [ ]:
# 
# Keep a list of dev keys to filter out
#
# This can also be done by passing through 'exclude' as an argument to generate_feature_table()
#
# Prerana 3uQeHE04bAOUjM8FIGnrCddmL7Y2
# Matt 2d4Hsxo6DDZsSgL5XlCRSHnlSRN2
# Luke qkE8zvjG9jQrjqLZh6E83qUmxXt2
# Nabiha ARMWazZdFHXXJQR46TLSoicFtLv2
#

exclude = ["3uQeHE04bAOUjM8FIGnrCddmL7Y2", "2d4Hsxo6DDZsSgL5XlCRSHnlSRN2", "qkE8zvjG9jQrjqLZh6E83qUmxXt2", "ARMWazZdFHXXJQR46TLSoicFtLv2"]

filter_accounts(d, exclude) # This function modifies the json argument directly, returns nothing. (also called inside generate_feature_table)

In [ ]:
# Create the feature table

df = generate_feature_table(d)

In [ ]:
articles_list = collect_articles(d)

In [ ]:
articles_list

In [ ]:
articles_df = pd.DataFrame([articles_list.keys(), articles_list.values()]).T

In [ ]:
# Export feature table to CSV file

today = date.today()

curr_date = today.strftime("%b_%d_%Y")

df.to_csv(f"feature_table_{curr_date}.csv") # name the file after today's date

files.download(f"feature_table_{curr_date}.csv")

In [ ]:
# Export articles df to CSV file

articles_df.to_csv("articles_list.csv")

files.download("articles_list.csv")